In [1]:
# importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
# function for getting book's old price. This is needed as some books dont have their old price, causing a problem when computer tries to get something that don't exist.
def old_price(notice): 
    try:
        book_old_price2 = notice.find_all("span", class_="value")[0].text.strip()
        return book_old_price2
    except IndexError:
        pass

In [3]:
# function for getting book's new price. This is needed as some books dont have their new price, causing a problem when computer tries to get something that don't exist.
def new_price(notice): 
    try:
        book_new_price2 = notice.find_all("span", class_="value")[1].text.strip()
        return book_new_price2
    except IndexError:
        pass

In [5]:
data=[] # data will be stored here.

for page in range(1,87,1): # There is 86 pages that needs to be scraped.

    url = "https://www.kitapyurdu.com/index.php?route=product/category&page="+str(page)+"&filter_category_all=true&path=1_41&filter_in_stock=1&sort=purchased_365&order=DESC&limit=100" # url of the website
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    page = requests.get(url, headers=headers).text # connect to website.
    soup = BeautifulSoup(page,"html.parser") # raw data.
    books = soup.find_all("div",class_="product-cr") # filtering the data.

    for book in books:
        item={}
        item["book_name"] = book.find("div", class_="name").text # gets the name of the book.
        item["book_publisher"] = book.find("div", class_="publisher").text # gets the name of the publisher.
        item["book_authors"] = book.find("div", class_="author").text # gets the name of the authors.
        item["book_old_price"] = old_price(book) # gets the old price of the book.
        item["book_new_price"] = new_price(book) # gets the new price of the book.
        book_rating = book.find("div", class_="rating").div["title"]
        item["book_reviews"]= [int(s) for s in re.findall(r'\d+', book_rating)] # gets the number of review for the book.
        data.append(item)

In [6]:
# Save the data to csv and xlsx
df = pd.DataFrame(data)
df.to_excel("kitapyurdu2.xlsx")
df.to_csv("kitapyurdu2.csv")